This will be the second prototype for the movies recommndation system. It will be made by making a content-based filtering algorithm.

Download the dataset from: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download

In [2]:
#Import libraries
import pandas as pd
import numpy as np

# nltk.download('punkt')
import nltk 
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix

import ipywidgets as widgets
from IPython.display import display

In [3]:
#Read files
movies = pd.read_csv("Datasets/movies_metadata.csv")

C:\Users\Sim\AppData\Local\Temp\ipykernel_11688\3178962405.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("Datasets/movies_metadata.csv")


In [4]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
#Examin the content
movies['overview'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [6]:
movies = movies.sample(20000).reset_index(drop=True)

In [7]:
print(movies['overview'].dtype)

object


In [8]:
#Have to turn objects into a string to tokenise and stem
movies['overview'] = movies['overview'].astype(str)

In [9]:
#Data Preprocessing
stemming = PorterStemmer()

#Apply tokanisation and stemming 
def token(txt):
    token = nltk.word_tokenize(txt)
    stemmed = [stemming.stem(i) for i in token]
    return " ".join(stemmed)

In [10]:
movies['overview'].apply(lambda x: token(x))

0        when a teenag girl move to england , with her ...
1        railroad fireman bill white is a carefre ladi ...
2        a savag stori of hate turn love and frenzi tur...
3        an ornithologist mistaken for an explos expert...
4        the continu of joe 's sexual dictat life delv ...
                               ...                        
19995    no one is safe when a killer , wield the deadl...
19996    haskel ( michael cain ) is assign a job by hi ...
19997    in thi delight anim tale , pardon-m pete ( bud...
19998    from the union squar theater in new york citi ...
19999    lestat de lioncourt is awaken from hi slumber ...
Name: overview, Length: 20000, dtype: object

In [11]:
#Vectorise the words to remove common english words
vectoriser = TfidfVectorizer(stop_words="english")
matrix = vectoriser.fit_transform(movies["overview"])

In [12]:
matrix_csr = csr_matrix(matrix)

In [13]:

# batch_size = 1000;
# num = matrix.shape[0]

# cosine_matrix = np.zeros((num,num))

# for i in range(0, num, batch_size):
#     start = i
#     end = min(i + batch_size, num)
    
#     batch_cosine = cosine_similarity(matrix[start:end], matrix)
    
#     cosine_matrix[start_idx:end_idx] = batch_cosine


#Compute pairwise cosine similarity
cosine_similarity = cosine_similarity(matrix_csr)


In [14]:
cosine_similarity[0]

array([1.        , 0.        , 0.        , ..., 0.00492757, 0.        ,
       0.        ])

In [15]:
#Recommender Algorithm
def recommender_system(movie, cosine_similarity):
    idx = movies[movies['original_title']==movie].index[0]
    distance = sorted(list(enumerate(cosine_similarity[idx])), reverse=True, key = lambda x:x[1])
    movie_output = []
    for m_id in distance[1:11]:
        movie_output.append(movies.iloc[m_id[0]].original_title)
    return movie_output

In [16]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 14,...",NaN,48358,tt0115820,en,The Canterville Ghost,"When a teenaged girl moves to England, with he...",...,1996-10-03,0.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Canterville Ghost,False,6.1,8.0
1,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,33001,tt0022236,en,Other Men's Women,Railroad fireman Bill White is a carefree ladi...,...,1931-01-17,0.0,71.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Romance of the Shining Rails,Other Men's Women,False,4.8,4.0
2,False,NaN,0,"[{'id': 37, 'name': 'Western'}]",NaN,153228,tt0046095,en,The Moonlighter,A savage story of hate turned love and frenzy ...,...,1953-09-19,0.0,77.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,THE MOST MAN-WOMAN EXCITEMENT EVENT TO EXPLODE...,The Moonlighter,False,4.5,3.0
3,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,17685,tt0060908,fr,Le roi de cœur,An ornithologist mistaken for an explosives ex...,...,1966-12-21,0.0,102.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,King of Hearts,False,6.8,15.0
4,False,"{'id': 258664, 'name': 'Nymphomaniac Collectio...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://www.magpictures.com/nymphomaniac/,249397,tt2382009,da,Nymphomaniac: Vol. II,The continuation of Joe's sexually dictated li...,...,2013-12-25,2227167.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Forget About Love,Nymphomaniac: Vol. II,False,6.6,820.0


In [17]:
recommender_system("The Avengers", cosine_similarity)

['Brink!',
 'Enemy Mine',
 'Shoulder Arms',
 'The Most Beautiful Thing',
 'Plastic',
 'Catching the Sun',
 'Le spie amano i fiori',
 'Demain',
 'Salt and Fire',
 'Inside']

In [18]:
# #Create a search function, for when entering title name
# def search_title(title):
#     vectored = vectorise.transform([title])
#     similar = cosine_similarity(vectored, matrix).flatten()
#     ten_most_similar_titles = np.argpartition(similar, -10)[-10:]
#     final = movie_file.iloc[ten_most_similar_titles][::-1]
#     return final

# #Add a easier way for user to enter media - from user survey
# title_input = widgets.Text(
#     value="",
#     description="Enter Title:",
#     disabled=False
# )

# output_title = widgets.Output()

# def when_typing(text):
#     with output_title:
#         output_title.clear_output()
#         title = text["new"]
#         if len(title) >= 3:
#             output_movie = search_title(title)
#             movie_ID = output_movie.iloc[0]["movieId"]
#             display(recommender_system(movie_ID))

# title_input.observe(when_typing, names="value")
# display(title_input, output_title)

In [19]:
def on_button_clicked(b):
    movie_name = text_input.value.strip()
    recommendations = recommender_system(movie_name, cosine_similarity)
    output.clear_output()
    with output:
        print("Recommendations:")
        for movie in recommendations:
            print(movie)

# Create text input box and button
text_input = widgets.Text(description='Enter Movie:', placeholder='Movie Name')
button = widgets.Button(description='Get Recommendations')

# Attach button click event handler
button.on_click(on_button_clicked)

# Create output widget to display recommendations
output = widgets.Output()

# Display widgets
display(text_input)
display(button)
display(output)

Text(value='', description='Enter Movie:', placeholder='Movie Name')

Button(description='Get Recommendations', style=ButtonStyle())

Output()